In [5]:
import os
import sys
os.chdir("/graspnet_ws/src/graspnet/pytorch_6dof-graspnet")
print(os.listdir())
sys.path.append("/graspnet_ws/src/graspnet/pytorch_6dof-graspnet")

['requirements.txt', 'README.md', '__pycache__', 'uniform_quaternions', 'demo', 'TRAINED_MODEL_LICENSE', 'shapenet_ids.txt', 'checkpoints', 'options', '.ipynb_checkpoints', 'eval.py', 'models', 'grasp_estimator.py', 'gripper_control_points', 'train.py', '.git', 'test.py', '.gitattributes', 'utils', 'gripper_models', 'renderer', 'data', 'LICENSE']


In [6]:
import numpy as np
import argparse
import glob
import mayavi.mlab as mlab

In [7]:
import grasp_estimator
from utils.visualization_utils import *
from utils import utils
from data import DataLoader

In [8]:
# parameters
class Args(object):
    def __init__(self):
        pass
args = Args()
# args.grasp_sampler_folder = 'checkpoints/gan_pretrained/'
args.grasp_sampler_folder = 'checkpoints/vae_pretrained'
args.grasp_evaluator_folder = 'checkpoints/evaluator_pretrained/'
# args.refinement_method = 'sampling'
args.refinement_method = 'gradient'
args.refine_steps = 10
args.npy_folder = 'demo/data/'
args.threshold = 0.8
args.choose_fn = 'better_than_threshold'
args.target_pc_size = 1024
args.num_grasp_samples = 10
args.generate_dense_grasps = False
args.batch_size = 30
args.train_data = False
if args.train_data:
    args.dataset_root_folder = ''

In [9]:
def backproject(depth_cv, intrinsic_matrix, return_finite_depth=True, return_selection=False):
    depth = depth_cv.astype(np.float32, copy=True)
    
    # get intrinsic matrix
    K = intrinsic_matrix
    Kinv = np.linalg.inv(K)
    
    #QIN compute the 3D points
    width = depth.shape[1]
    height = depth.shape[0]
    
    # construct the 2D points matrix
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    ones = np.ones((height, width), dtype=np.float32)
    x2d = np.stack((x,y,ones), axis=2).reshape(width*height,3)
    
    # backprojection
    R = np.dot(Kinv, x2d.transpose())
    
    # compute the 3D points
    X = np.multiply(np.tile(depth.reshape(1, width*height), (3,1)), R)
    X = np.array(X).transpose()
    if return_finite_depth:
        selection = np.isfinite(X[:,0])
        X = X[selection, :]
    
    if return_selection:
        return X, selection
    
    return X
    

In [10]:
# processing
grasp_sampler_args = utils.read_checkpoint_args(args.grasp_sampler_folder)
grasp_sampler_args.is_train = False
#print(grasp_sampler_args)
grasp_evaluator_args = utils.read_checkpoint_args(args.grasp_evaluator_folder)
grasp_evaluator_args.continue_train = True
#print(grasp_evaluator_args)
estimator = grasp_estimator.GraspEstimator(grasp_sampler_args, grasp_evaluator_args, args)

loading the model from ./checkpoints/evaluator_pretrained/latest_net.pth
loading the model from ./checkpoints/vae_pretrained/latest_net.pth


In [8]:
for npy_files in glob.glob(os.path.join(args.npy_folder, '*.npy')):
    # Depending on your numpy version you may need to change allow_pickle
    # from True to False

    data = np.load(npy_file, allow_pickle=True, encoding='latin1').item()

    depth = data['depth']
    image = data['image']
    K = data['intrinsics_matrix']
    # Removing points that are farther than 1 meter or missing depth values
    # depth[depth == 0 or depth > 1] = np.nan

    np.nan_to_num(depth, copy=False)
    mask = np.where(np.logical_or(depth==0, depth>1))
    depth[mask] = np.nan
    pc, selection = backproject(depth, K, return_finite_depth=True, return_selection=True)
    pc_colors = image.copy()
    pc_colors = np.reshape(pc_colors, [-1,3])
    pc_colors = pc_colors[selection, :]

    # Smoothed pc comes from averaging the depth for 10 frames and removing the pixels with jittery depth
    object_pc = data['smoothed_object_pc']
    generated_grasps, generated_scores = estimator.generate_and_refine_grasps(object_pc)
    mlab.figure(bgcolor=(1,1,1))
    draw_scene(pc, pc_color=pc_colors, grasps=generated_grasps, grasp_scores=generated_scores)
    print('close the window to continue to next object ...')
    mlab.show()
        

loading the model from ./checkpoints/evaluator_pretrained/latest_net.pth
loading the model from ./checkpoints/vae_pretrained/latest_net.pth


In [11]:
data = np.load('demo/data/blue_mug.npy', allow_pickle=True, encoding='latin1').item()
depth = data['depth']
image = data['image']
print(depth.shape)
print(image.shape)
K = data['intrinsics_matrix']
print(K)

np.nan_to_num(depth, copy=False)
mask = np.where(np.logical_or(depth == 0, depth > 1))

(480, 640)
(480, 640, 3)
[[616.36529541   0.         310.25881958]
 [  0.         616.20294189 236.59980774]
 [  0.           0.           1.        ]]


In [12]:
depth[mask] = np.nan
pc, selection = backproject(depth, K, return_finite_depth=True, return_selection=True)
pc_colors = image.copy()
pc_colors = np.reshape(pc_colors, [-1,3])
pc_colors = pc_colors[selection, :]

In [13]:
object_pc = data['smoothed_object_pc']

In [14]:
generated_grasps, generated_scores = estimator.generate_and_refine_grasps(object_pc)

choose_better_than_threshold threshold= 0.8


In [18]:
print(generated_grasps[0])

[[-0.66426615  0.03239556 -0.74679382  0.10982121]
 [-0.49010772  0.73546612  0.46785042 -0.05234753]
 [ 0.56439783  0.67678661 -0.47266793  0.55081968]
 [ 0.          0.          0.          1.        ]]


In [37]:
print(generated_scores)

[0.9405957460403442, 0.994360625743866, 0.8906962275505066, 0.9056081175804138, 0.8103220462799072, 0.8224443793296814, 0.8014820218086243, 0.8937219381332397, 0.8477743268013, 0.9333900809288025, 0.9266433119773865, 0.9471819400787354, 0.8828741908073425, 0.8290473818778992, 0.9028867483139038, 0.9067367315292358, 0.8130753636360168, 0.8888319730758667, 0.8559100031852722, 0.9273698329925537, 0.8253844976425171, 0.9181490540504456, 0.8258219957351685, 0.9201022386550903, 0.8364370465278625]


In [20]:
from scipy.spatial.transform import Rotation as R
rotation_matrix = [[generated_grasps[0][0,0], generated_grasps[0][0,1], generated_grasps[0][0,2]],
                   [generated_grasps[0][1,0], generated_grasps[0][1,1], generated_grasps[0][1,2]],
                   [generated_grasps[0][2,0], generated_grasps[0][2,1], generated_grasps[0][2,2]]]
r = R.from_matrix(rotation_matrix)
quat = r.as_quat()
print(quat[0])

-0.13503302203500767
